# ОИАД — Лабораторная работа №3  
**Medical Insurance Cost Dataset**

**Цель работы:** подготовить данные, построить многомерную линейную регрессию (аналитически и численно), добавить регуляризацию (Ridge), и оценить обобщающую способность моделей по MSE.

**Файл датасета:**  
`insurance_train.csv`  
`insurance_test.csv`

**Источник:**  
[Medical Insurance Cost Dataset (Kaggle)](https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset?resource=download)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Загружаем данные
train = pd.read_csv("../datasets/insurance_train.csv")
test = pd.read_csv("../datasets/insurance_test.csv")

# Просмотр структуры
train.head()


,age,sex,bmi,children,smoker,region,charges
0,26,male,27.060,0,yes,southeast,17043.34140
1,58,male,36.955,2,yes,northwest,47496.49445
2,20,female,24.420,0,yes,southeast,26125.67477
3,51,female,38.060,0,yes,southeast,44400.40640
4,62,female,25.000,0,no,southwest,13451.12200


## 1. Подготовка данных

1. Проверить наличие пропусков и выбросов  
2. Привести категориальные признаки к числовым  
3. Вычислить парные корреляции признаков

### Типы признаков в наборе данных Insurance

| Признак        | Тип признака                | Описание |
|----------------|----------------------------|----------|
| `age`          | Количественный/непрерывный | Возраст (целое число) |
| `sex`          | Бинарный                    | Пол (мужчина, женщина) |
| `bmi`          | Количественный/непрерывный | Индекс массы тела — показатель содержания жира в организме на основе роста и веса (плавающий) |
| `children`     | Количественный/дискретный  | Количество детей, охваченных медицинской страховкой (целое число) |
| `smoker`       | Бинарный                    | Статус курения (да/нет) |
| `region`       | Категориальный (номинальный)| Жилой регион в США (northwest, northeast, southeast, southwest) |
| `charges`      | Количественный/непрерывный | Стоимость выставленной медицинской страховки (плавающая сумма) |


- Количественные признаки (`age`, `bmi`, `children`, `charges`) можно использовать напрямую для построения регрессии.  
- Бинарные признаки (`sex`, `smoker`) можно закодировать как 0 и 1.  
- Категориальные признаки (`region`) преобразуются через **one-hot encoding** — отдельный столбец для каждого региона.  
- Порядковые признаки здесь отсутствуют, но обычно их можно просто пронумеровать, сохраняя порядок.



In [26]:
# Проверим пропуски
print(train.isnull().sum())

# Проверим выбросы по BMI и charges с помощью z-score
z_scores = np.abs(stats.zscore(train.select_dtypes(include=np.number)))
print("Количество выбросов:", (z_scores > 3).sum().sum())

train_prep = train.copy()
test_prep = test.copy()

# Бинарные замены
for df in [train_prep, test_prep]:
    df["sex"] = df["sex"].map({"male": 1, "female": 0})
    df["smoker"] = df["smoker"].map({"yes": 1, "no": 0})

# One-hot кодирование региона
train_prep = pd.get_dummies(train_prep, columns=["region"], drop_first=True)
test_prep = pd.get_dummies(test_prep, columns=["region"], drop_first=True)

train_prep.head()


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64
Количество выбросов: 9


,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,26,1,27.060,0,1,17043.34140,False,True,False
1,58,1,36.955,2,1,47496.49445,True,False,False
2,20,0,24.420,0,1,26125.67477,False,True,False
3,51,0,38.060,0,1,44400.40640,False,True,False
4,62,0,25.000,0,0,13451.12200,False,False,True


### Корреляция признаков

Корреляция показывает силу линейной зависимости между переменными.

$$
r(x, y) = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum(x_i - \bar{x})^2 \sum(y_i - \bar{y})^2}}
$$

Значения $ r \in [-1, 1] $:  
- $ r > 0 $: прямая связь  
- $ r < 0 $: обратная  
- $ |r| \approx 1 $: сильная зависимость  
- $ |r| \approx 0 $: отсутствует


In [15]:
corr = train_prep.corr(numeric_only=True)
corr["charges"].sort_values(ascending=False)


charges             1.000000
smoker              0.783519
age                 0.298395
bmi                 0.219566
children            0.069444
sex                 0.060221
region_southeast    0.009792
region_northwest   -0.032287
region_southwest   -0.053905
Name: charges, dtype: float64

## 2. Многомерная линейная регрессия

### Модель регрессии
$$
f(x, w) = \sum_{i=1}^{n} w_i x_i = Xw
$$

Функционал потерь
$$
Q(w) = \sum_{i=1}^{\ell} (f(x_i, w) - y_i)^2 = ||Xw - y||^2 \rightarrow \min_{w} 
$$

### Аналитическое решение

Для поиска точки минимума, приравниваем градиент к нулю.
$$
2X^T(Xw-y) = 0
$$

$$
X^TXw = X^Ty
$$

$$
w^* = (X^TX)^{-1}X^Ty
$$

### Численное решение

 Основная идея состоит в итерационном движении от одной точки к другой в пространстве параметров модели. Направление движения определяется с помощью градиента функции. Градиент функции в некоторой точке указывает направление наискорейшего роста. Мы же хотим минимизировать функцию, поэтому направление выберем как минус градиент в этой точке. Размер шага будет определятся абсолютным значением градиента в точке и некоторым параметром.

In [20]:
# Преобразуем всё в float
train_prep = train_prep.astype(float)
test_prep = test_prep.astype(float)

# Формируем матрицу X и вектор y
X_train = train_prep.drop(columns=["charges"]).to_numpy()
y_train = train_prep["charges"].to_numpy().reshape(-1, 1)

# Добавляем столбец единиц для свободного члена
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

# Аналитическое решение
w_analytical = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
w_analytical[:5]


array([[-11458.00761083],
       [   256.71070712],
       [  -655.40504609],
       [   350.86390876],
       [   483.10419606]])

**Метод градиентного спуска**

Классический метод градиентного спуска. Другие методы модифицируют его, внося изменения в подход к определению размеру или направлению шага.

$w^{(0)}$ - начальное приближение

$\lambda$ - размер градиентного шага

$w^{(k+1)} = w^{(k)} - \lambda \cdot \frac{\partial Q}{\partial w}|_{w=w^{(k)}}$

Остановка просходит в случаях:
* градиент близок к нулю
* изменение параметров близко к нулю
* достигнуто ограничительное число итераций
  
**Стохастический градиентный спуск**

Идея состоит в расчете не точного значения градиента $\frac{\partial Q}{\partial w}|_{w=w^{(k)}}$ на всей выборке $X$ размера $\ell$, а оценке его значения по части выборки $\tilde{X} \subset X$ меньшего размера $\tilde{\ell} < \ell$.


In [27]:
# Градиентный спуск
def gradient_descent(X, y, lr=1e-7, n_iter=10000, tol=1e-6):
    w = np.zeros((X.shape[1], 1))
    for i in range(n_iter):
        grad = 2 * X.T @ (X @ w - y)
        w_new = w - lr * grad
        if np.linalg.norm(w_new - w) < tol:
            break
        w = w_new
    return w

w_gd = gradient_descent(X_train, y_train)
w_gd[:5]


array([[-184.00095262],
       [ 212.51330928],
       [ 157.39687464],
       [ 179.04508277],
       [ 204.20654496]])

## Регуляризация

Когда признаки сильно коррелированы, матрица $(X^TX)^{-1}$ становится неустойчивой, что приводит к большим значениям весов $w$ и нестабильной модели. Чтобы уменьшить этот эффект, добавляют регуляризационный член в функционал ошибки.

### Гребневая регрессия (Ridge, L2)
Функционал ошибки:
$$
Q_{L_2}(w) = ||Xw - y||^2 + \alpha ||w||_2^2 \rightarrow \min_{w}
$$
где
$$
||w||_2 = \sqrt{\sum_{i=1}^{n} w_i^2}
$$

Аналитическое решение:
$$
w^* = (X^TX + \alpha I)^{-1} X^T y
$$
где $I$ — единичная матрица, $\alpha$ — параметр регуляризации.

Градиент для численной оптимизации:
$$
\frac{\partial Q_{L_2}}{\partial w} = 2 X^T (Xw - y) + 2 \alpha w
$$

### LASSO (L1)
Функционал:
$$
Q_{L_1}(w) = ||Xw - y||^2 + \beta ||w||_1 \rightarrow \min_{w}
$$
$$
||w||_1 = \sum_{i=1}^{n} |w_i|
$$

Эта задача не имеет аналитического решения, поэтому обычно решается только численно.


In [46]:
def ridge_regression(X, y, alpha=1.0):
    n_features = X.shape[1]
    I = np.eye(n_features)
    w = np.linalg.inv(X.T @ X + alpha * I) @ X.T @ y
    return w

w_ridge = ridge_regression(X_train, y_train, alpha=100)
w_ridge[:5]


array([[-942.45129577],
       [ 211.93612819],
       [ 119.43930264],
       [ 164.95858935],
       [ 280.56963267]])

## 4. Оценка обобщающей способности модели

Для оценки качества используем **среднеквадратичную ошибку (MSE)**:
$$
MSE = \frac{1}{n}\sum_{i=1}^n (y_i - \hat{y_i})^2
$$

Сравним три модели:
1. Константную (прогноз средним значением)  
2. Линейную (из пункта 2)  
3. Регуляризованную (из пункта 3)


In [47]:
# Подготовим тестовые данные
X_test = test_prep.drop(columns=["charges"]).to_numpy()
y_test = test_prep["charges"].to_numpy().reshape(-1, 1)
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# Предсказания
y_pred_const = np.full_like(y_test, np.mean(y_train))
y_pred_lin = X_test @ w_analytical
y_pred_ridge = X_test @ w_ridge

# Функция MSE
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

print("MSE (const):", mse(y_test, y_pred_const))
print("MSE (linear):", mse(y_test, y_pred_lin))
print("MSE (ridge):", mse(y_test, y_pred_ridge))


MSE (const): 141830094.35903943
MSE (linear): 34216008.75883003
MSE (ridge): 72996049.63473104


##  Результаты

| Модель          | MSE (чем меньше, тем лучше) |
|-----------------|-----------------------------:|
| Константная     | 141 830 094.36              |
| Линейная        | **34 216 008.76**           |
| Ridge-регрессия | 72 996 049.63               |


## Анализ и выводы

1. **Константная модель**  
   Использует простейший прогноз — всегда предсказывает среднее значение `charges` по обучающей выборке.  
   MSE = 141 млн — высокая ошибка, т.к. модель не учитывает никаких признаков и не улавливает зависимости.

2. **Обычная линейная регрессия (аналитически/градиентно)**  
   Ошибка резко уменьшается — MSE ≈ 34 млн.  
   Это говорит о том, что модель **хорошо улавливает взаимосвязь** между признаками (особенно `smoker`, `age`, `bmi`) и итоговой стоимостью страховки.  
   Основное влияние вносит бинарный признак `smoker` (коэффициент корреляции 0.78).

3. **Ridge-регрессия (регуляризация L2)**  
   Ошибка выросла (MSE ≈ 73 млн).  
   Это объясняется тем, что регуляризация **снижает веса модели**, чтобы уменьшить переобучение, но при слишком сильном коэффициенте `α` — модель теряет точность.  
   В данном случае — признаков немного, мультиколлинеарность слабая, поэтому добавление регуляризации **не дало улучшения**, а только немного ухудшило результат.


## Итоговый вывод

На данном наборе данных:
- **наилучший результат** показала **обычная многомерная линейная регрессия** без регуляризации;  
- **регуляризация** полезна при сильной корреляции между признаками или большом количестве параметров,  
  но здесь — признаки независимы, поэтому Ridge лишь ограничил веса, ухудшив точность.

Таким образом, **линейная модель наиболее адекватно описывает зависимость между возрастом, курением, индексом массы тела и стоимостью страховки**.

